In [80]:
from __future__ import division
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import operator










0it [00:00, ?it/s]


In [2]:
with open("train.csv", "r") as f:
    train_text = f.read()
    
rows = train_text.split("\n")
rows_list = []

for i in rows:
    
    data = i.split("|")
    
    if len(data) == 3:
        user_id = data[0]
        movie_name = data[1]
        watch_time = float(data[2])
    
        rows_list.append([user_id, movie_name, watch_time])
        
train = pd.DataFrame(rows_list)
train.columns = ["user_id", "title", "watch_time"]

test = pd.read_csv("test.csv")

with open("movies_meta.csv", "r") as f:
    movies_meta = f.read()

movies = movies_meta.split("\n")[1:]

movies_df = pd.DataFrame()

movies_list = []

for movie in movies:
    
    data = movie.split("|")

    if len(data) == 4:

        movie_name = data[0]
        genre = data[1]

        langs = data[2].split(",")

        movie_length = float(data[3])

        for lang in langs:
            movies_list.append([movie_name, genre, lang, movie_length])

movies_df = pd.DataFrame(movies_list)
movies_df.columns = ["title", "movie_genre", "movie_language", "movie_length"]    

In [52]:
train_mod = pd.merge(train, movies_df, on="title")
train_mod["watch_ratio"] = train_mod["watch_time"]/train_mod["movie_length"]

In [136]:
title_count = {}

def get_movie_counts(row):
    
#     lang_genre_key = row.movie_language + "_" + row.movie_genre
    for lang_genre_key in [row.movie_language, row.movie_genre]:

        if lang_genre_key not in title_count:
            title_count[lang_genre_key] = {}

        if row.title not in title_count[lang_genre_key]:
            title_count[lang_genre_key][row.title] = 1
        else:
            title_count[lang_genre_key][row.title] += 1        
        
train_mod.progress_apply(get_movie_counts, axis = 1)












  0%|          | 0/2705037 [00:00<?, ?it/s]










  0%|          | 1/2705037 [00:01<917:44:45,  1.22s/it]










  0%|          | 1087/2705037 [00:01<642:11:05,  1.17it/s]










  0%|          | 2209/2705037 [00:01<449:21:46,  1.67it/s]










  0%|          | 3387/2705037 [00:01<314:26:10,  2.39it/s]










  0%|          | 4535/2705037 [00:01<220:01:53,  3.41it/s]










  0%|          | 5711/2705037 [00:01<153:58:26,  4.87it/s]










  0%|          | 6847/2705037 [00:01<107:45:22,  6.96it/s]










  0%|          | 7948/2705037 [00:01<75:25:08,  9.93it/s] 










  0%|          | 8987/2705037 [00:02<52:47:42, 14.19it/s]










  0%|          | 10018/2705037 [00:02<36:58:04, 20.25it/s]










  0%|          | 11196/2705037 [00:02<25:53:07, 28.91it/s]










  0%|          | 12255/2705037 [00:02<18:08:01, 41.25it/s]










  0%|          | 13358/2705037 [00:02<12:42:31, 58.83it/s]










  1%|          | 14423/2705037 [00:02<8:54

  3%|▎         | 91686/2705037 [00:09<03:53, 11197.22it/s]










  3%|▎         | 92810/2705037 [00:09<03:53, 11207.27it/s]










  3%|▎         | 94017/2705037 [00:09<03:48, 11450.80it/s]










  4%|▎         | 95165/2705037 [00:09<03:52, 11226.56it/s]










  4%|▎         | 96325/2705037 [00:09<03:50, 11330.63it/s]










  4%|▎         | 97460/2705037 [00:09<03:53, 11187.69it/s]










  4%|▎         | 98693/2705037 [00:09<03:46, 11504.85it/s]










  4%|▎         | 99847/2705037 [00:10<03:47, 11444.87it/s]










  4%|▎         | 100994/2705037 [00:10<03:48, 11401.85it/s]










  4%|▍         | 102197/2705037 [00:10<03:44, 11581.49it/s]










  4%|▍         | 103358/2705037 [00:10<03:51, 11253.61it/s]










  4%|▍         | 104487/2705037 [00:10<03:54, 11101.55it/s]










  4%|▍         | 105672/2705037 [00:10<03:50, 11298.24it/s]










  4%|▍         | 106805/2705037 [00:10<03:53, 11133.01it/s]










  4%|▍         | 107921/2705

  7%|▋         | 183656/2705037 [00:17<03:40, 11454.23it/s]










  7%|▋         | 184807/2705037 [00:17<03:47, 11076.90it/s]










  7%|▋         | 185921/2705037 [00:17<03:50, 10908.75it/s]










  7%|▋         | 187020/2705037 [00:17<03:50, 10929.51it/s]










  7%|▋         | 188216/2705037 [00:17<03:44, 11218.54it/s]










  7%|▋         | 189344/2705037 [00:18<03:43, 11236.85it/s]










  7%|▋         | 190519/2705037 [00:18<03:40, 11385.37it/s]










  7%|▋         | 191661/2705037 [00:18<03:43, 11233.88it/s]










  7%|▋         | 192787/2705037 [00:18<03:45, 11130.69it/s]










  7%|▋         | 194046/2705037 [00:18<03:37, 11531.53it/s]










  7%|▋         | 195205/2705037 [00:18<03:41, 11322.10it/s]










  7%|▋         | 196342/2705037 [00:18<03:45, 11109.20it/s]










  7%|▋         | 197596/2705037 [00:18<03:38, 11501.46it/s]










  7%|▋         | 198753/2705037 [00:18<03:44, 11167.07it/s]










  7%|▋         | 199

 10%|█         | 276861/2705037 [00:25<03:31, 11503.42it/s]










 10%|█         | 278080/2705037 [00:25<03:27, 11700.66it/s]










 10%|█         | 279254/2705037 [00:25<03:28, 11631.98it/s]










 10%|█         | 280420/2705037 [00:26<03:32, 11423.77it/s]










 10%|█         | 281618/2705037 [00:26<03:29, 11581.79it/s]










 10%|█         | 283028/2705037 [00:26<03:17, 12234.43it/s]










 11%|█         | 284263/2705037 [00:26<03:25, 11785.39it/s]










 11%|█         | 285454/2705037 [00:26<03:30, 11502.57it/s]










 11%|█         | 286716/2705037 [00:26<03:24, 11814.06it/s]










 11%|█         | 287907/2705037 [00:26<03:24, 11839.89it/s]










 11%|█         | 289098/2705037 [00:26<03:26, 11712.64it/s]










 11%|█         | 290274/2705037 [00:26<03:29, 11544.32it/s]










 11%|█         | 291464/2705037 [00:26<03:27, 11644.12it/s]










 11%|█         | 292632/2705037 [00:27<03:32, 11356.28it/s]










 11%|█         | 293

 14%|█▎        | 369938/2705037 [00:33<03:25, 11353.48it/s]










 14%|█▎        | 371241/2705037 [00:33<03:17, 11808.48it/s]










 14%|█▍        | 372488/2705037 [00:34<03:14, 11998.06it/s]










 14%|█▍        | 373694/2705037 [00:34<03:16, 11863.10it/s]










 14%|█▍        | 374885/2705037 [00:34<03:20, 11611.62it/s]










 14%|█▍        | 376051/2705037 [00:34<03:27, 11236.47it/s]










 14%|█▍        | 377342/2705037 [00:34<03:19, 11688.71it/s]










 14%|█▍        | 378575/2705037 [00:34<03:15, 11872.03it/s]










 14%|█▍        | 379769/2705037 [00:34<03:17, 11795.04it/s]










 14%|█▍        | 380954/2705037 [00:34<03:18, 11694.02it/s]










 14%|█▍        | 382127/2705037 [00:34<03:21, 11507.63it/s]










 14%|█▍        | 383320/2705037 [00:35<03:19, 11631.13it/s]










 14%|█▍        | 384486/2705037 [00:35<03:23, 11394.13it/s]










 14%|█▍        | 385629/2705037 [00:35<03:27, 11183.38it/s]










 14%|█▍        | 386

...

 88%|████████▊ | 2390404/2705037 [03:32<00:27, 11339.77it/s]










 88%|████████▊ | 2391542/2705037 [03:32<00:28, 11125.21it/s]










 88%|████████▊ | 2392798/2705037 [03:32<00:27, 11519.48it/s]










 89%|████████▊ | 2394029/2705037 [03:32<00:26, 11744.65it/s]










 89%|████████▊ | 2395209/2705037 [03:32<00:27, 11304.99it/s]










 89%|████████▊ | 2396360/2705037 [03:32<00:27, 11364.81it/s]










 89%|████████▊ | 2397502/2705037 [03:32<00:27, 11284.31it/s]










 89%|████████▊ | 2398635/2705037 [03:32<00:27, 11046.11it/s]










 89%|████████▊ | 2399744/2705037 [03:33<00:28, 10876.98it/s]










 89%|████████▉ | 2400835/2705037 [03:33<00:28, 10725.59it/s]










 89%|████████▉ | 2401972/2705037 [03:33<00:27, 10910.47it/s]










 89%|████████▉ | 2403209/2705037 [03:33<00:26, 11310.50it/s]










 89%|████████▉ | 2404346/2705037 [03:33<00:27, 11037.38it/s]










 89%|████████▉ | 2405467/2705037 [03:33<00:27, 11087.26it/s]










 89%|█

 92%|█████████▏| 2481331/2705037 [03:40<00:19, 11442.54it/s]










 92%|█████████▏| 2482527/2705037 [03:40<00:19, 11592.58it/s]










 92%|█████████▏| 2483688/2705037 [03:40<00:19, 11310.89it/s]










 92%|█████████▏| 2484822/2705037 [03:40<00:19, 11296.64it/s]










 92%|█████████▏| 2485990/2705037 [03:40<00:19, 11407.01it/s]










 92%|█████████▏| 2487133/2705037 [03:40<00:19, 11408.15it/s]










 92%|█████████▏| 2488275/2705037 [03:40<00:19, 11202.95it/s]










 92%|█████████▏| 2489434/2705037 [03:41<00:19, 11315.36it/s]










 92%|█████████▏| 2490667/2705037 [03:41<00:18, 11599.39it/s]










 92%|█████████▏| 2491830/2705037 [03:41<00:18, 11229.74it/s]










 92%|█████████▏| 2493100/2705037 [03:41<00:18, 11633.37it/s]










 92%|█████████▏| 2494271/2705037 [03:41<00:18, 11624.49it/s]










 92%|█████████▏| 2495439/2705037 [03:41<00:18, 11506.23it/s]










 92%|█████████▏| 2496594/2705037 [03:41<00:18, 11099.07it/s]










 92%|█

 95%|█████████▌| 2570787/2705037 [03:48<00:11, 11446.16it/s]










 95%|█████████▌| 2571936/2705037 [03:48<00:11, 11167.42it/s]










 95%|█████████▌| 2573117/2705037 [03:48<00:11, 11348.89it/s]










 95%|█████████▌| 2574256/2705037 [03:48<00:11, 11180.33it/s]










 95%|█████████▌| 2575378/2705037 [03:48<00:11, 10970.85it/s]










 95%|█████████▌| 2576496/2705037 [03:48<00:11, 11031.95it/s]










 95%|█████████▌| 2577776/2705037 [03:48<00:11, 11507.34it/s]










 95%|█████████▌| 2578934/2705037 [03:49<00:11, 11429.16it/s]










 95%|█████████▌| 2580082/2705037 [03:49<00:11, 11150.11it/s]










 95%|█████████▌| 2581202/2705037 [03:49<00:11, 11146.30it/s]










 95%|█████████▌| 2582321/2705037 [03:49<00:11, 11067.62it/s]










 96%|█████████▌| 2583542/2705037 [03:49<00:10, 11385.26it/s]










 96%|█████████▌| 2584685/2705037 [03:49<00:10, 11370.54it/s]










 96%|█████████▌| 2585825/2705037 [03:49<00:10, 11182.24it/s]










 96%|█

 98%|█████████▊| 2662009/2705037 [03:56<00:03, 11687.04it/s]










 98%|█████████▊| 2663183/2705037 [03:56<00:03, 11335.91it/s]










 98%|█████████▊| 2664361/2705037 [03:56<00:03, 11465.28it/s]










 99%|█████████▊| 2665677/2705037 [03:56<00:03, 11925.12it/s]










 99%|█████████▊| 2666878/2705037 [03:56<00:03, 11639.25it/s]










 99%|█████████▊| 2668049/2705037 [03:56<00:03, 11593.00it/s]










 99%|█████████▊| 2669264/2705037 [03:56<00:03, 11750.61it/s]










 99%|█████████▊| 2670443/2705037 [03:57<00:02, 11551.40it/s]










 99%|█████████▉| 2671608/2705037 [03:57<00:02, 11580.53it/s]










 99%|█████████▉| 2672769/2705037 [03:57<00:02, 11319.92it/s]










 99%|█████████▉| 2673905/2705037 [03:57<00:02, 11256.30it/s]










 99%|█████████▉| 2675093/2705037 [03:57<00:02, 11435.93it/s]










 99%|█████████▉| 2676239/2705037 [03:57<00:02, 11310.49it/s]










 99%|█████████▉| 2677409/2705037 [03:57<00:02, 11422.83it/s]










 99%|█

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
2705007    None
2705008    None
2705009    None
2705010    None
2705011    None
2705012    None
2705013    None
2705014    None
2705015    None
2705016    None
2705017    None
2705018    None
2705019    None
2705020    None
2705021    None
2705022    None
2705023    None
2705024    None
2705025    None
2705026    None
2705027    None
2705028    None
2705029    None
2705030    None
2705031    None
2705032    None
2705033    None
2705034    None
2705035    None
2705036    None
Length: 2705037, dtype: 

In [137]:
popular_lang_genre_movies = {}

for lang_genre_key in title_count:
    popular_lang_genre_movies[lang_genre_key] = [i[0] for i in sorted(title_count[lang_genre_key].items(), key=operator.itemgetter(1), reverse=True)[:20]]

In [138]:
popular_lang_genre_movies

{'action': ['kabali',
  'janatha garage',
  'bang bang',
  'dishoom',
  'baahubali - the beginning',
  'khaidi no. 150',
  'guru',
  'baby',
  'action hero biju',
  'kick',
  'holiday',
  'chingari',
  'action jackson',
  'pulimurugan',
  'gautamiputra satakarni',
  'daagdi chawl',
  'singham returns',
  'kammattipadam',
  'sardar gabbar singh',
  'maari'],
 'bengali': ['hero',
  'premer kahini',
  'love express',
  'kelor kirti',
  'haripada bandwala',
  'guru',
  'besh korechi prem korechi',
  'parbona ami charte toke',
  'shudhu tomari jonya',
  'ki kore toke bolbo',
  'jamai 420',
  'power',
  'zulfiqar',
  'rajkahini',
  'awara',
  'herogiri',
  'deewana',
  'sathi',
  'borbaad',
  'abar byomkesh'],
 'comedy': ['jolly llb',
  'housefull 3',
  'yaariyan',
  'dolly ki doli',
  'love express',
  'housefull 2',
  'kelor kirti',
  '2 countries',
  'bhagam bhag',
  'adi kapyare kootamani',
  'khatta meetha',
  'golmaal 3',
  'marubhoomiyile aana',
  'aadu',
  'bade miyan chote miyan',
 

In [139]:
user_lang_genre = {}
# user_lang_genre_count = {}

def get_user_lang_genre(row):
    
#     lang_genre_key = row.movie_language + "_" + row.movie_genre
    for lang_genre_key in [row.movie_language, row.movie_genre]:

        if row.user_id not in user_lang_genre:
            user_lang_genre[str(row.user_id)] = {}
    #         user_lang_genre_count[str(row.user_id)] = {}

        if lang_genre_key not in user_lang_genre[str(row.user_id)]:

            user_lang_genre[str(row.user_id)][lang_genre_key] = row.watch_time

    #         user_lang_genre[str(row.user_id)][lang_genre_key] = float(row.watch_ratio)
    #         user_lang_genre_count[str(row.user_id)][lang_genre_key] = 1

        else:
            user_lang_genre[str(row.user_id)][lang_genre_key] += row.watch_time        

    #         user_lang_genre[str(row.user_id)][lang_genre_key] += float(row.watch_ratio)        
    #         user_lang_genre_count[str(row.user_id)][lang_genre_key] += 1        
    
train_mod.progress_apply(get_user_lang_genre, axis = 1)

# for user in user_lang_genre:
    
#     for lang_genre in user_lang_genre[user]:
        
#         user_lang_genre[user][lang_genre] /= user_lang_genre_count[user][lang_genre]        












  0%|          | 0/2705037 [00:00<?, ?it/s]










  0%|          | 1/2705037 [00:01<910:35:25,  1.21s/it]










  0%|          | 582/2705037 [00:01<637:18:54,  1.18it/s]










  0%|          | 1301/2705037 [00:01<446:02:00,  1.68it/s]










  0%|          | 1924/2705037 [00:01<312:11:15,  2.41it/s]










  0%|          | 2577/2705037 [00:01<218:30:46,  3.44it/s]










  0%|          | 3199/2705037 [00:01<152:57:36,  4.91it/s]










  0%|          | 3826/2705037 [00:01<107:04:59,  7.01it/s]










  0%|          | 4437/2705037 [00:01<74:58:41, 10.01it/s] 










  0%|          | 5144/2705037 [00:02<52:30:10, 14.28it/s]










  0%|          | 5824/2705037 [00:02<36:46:32, 20.39it/s]










  0%|          | 6458/2705037 [00:02<25:46:21, 29.09it/s]










  0%|          | 7118/2705037 [00:02<18:04:13, 41.47it/s]










  0%|          | 7837/2705037 [00:02<12:40:38, 59.10it/s]










  0%|          | 8515/2705037 [00:02<8:54:18, 8

  2%|▏         | 55035/2705037 [00:09<06:33, 6733.37it/s]










  2%|▏         | 55710/2705037 [00:09<06:40, 6621.30it/s]










  2%|▏         | 56374/2705037 [00:09<06:42, 6581.95it/s]










  2%|▏         | 57034/2705037 [00:09<06:45, 6536.52it/s]










  2%|▏         | 57791/2705037 [00:09<06:28, 6814.79it/s]










  2%|▏         | 58477/2705037 [00:09<06:34, 6702.30it/s]










  2%|▏         | 59151/2705037 [00:10<06:40, 6604.64it/s]










  2%|▏         | 59842/2705037 [00:10<06:35, 6691.90it/s]










  2%|▏         | 60514/2705037 [00:10<06:38, 6633.32it/s]










  2%|▏         | 61179/2705037 [00:10<06:41, 6592.17it/s]










  2%|▏         | 61840/2705037 [00:10<06:44, 6527.93it/s]










  2%|▏         | 62519/2705037 [00:10<06:40, 6602.57it/s]










  2%|▏         | 63211/2705037 [00:10<06:34, 6692.99it/s]










  2%|▏         | 63882/2705037 [00:10<06:37, 6649.06it/s]










  2%|▏         | 64598/2705037 [00:10<06:28, 679

  4%|▍         | 110757/2705037 [00:17<06:44, 6408.82it/s]










  4%|▍         | 111413/2705037 [00:17<06:42, 6451.70it/s]










  4%|▍         | 112091/2705037 [00:17<06:36, 6546.37it/s]










  4%|▍         | 112747/2705037 [00:18<06:42, 6441.62it/s]










  4%|▍         | 113414/2705037 [00:18<06:38, 6506.84it/s]










  4%|▍         | 114070/2705037 [00:18<06:37, 6521.49it/s]










  4%|▍         | 114723/2705037 [00:18<06:38, 6495.65it/s]










  4%|▍         | 115479/2705037 [00:18<06:21, 6779.13it/s]










  4%|▍         | 116194/2705037 [00:18<06:16, 6885.21it/s]










  4%|▍         | 116953/2705037 [00:18<06:05, 7081.87it/s]










  4%|▍         | 117665/2705037 [00:18<06:13, 6924.38it/s]










  4%|▍         | 118403/2705037 [00:18<06:06, 7052.76it/s]










  4%|▍         | 119112/2705037 [00:19<06:17, 6842.86it/s]










  4%|▍         | 119844/2705037 [00:19<06:10, 6975.19it/s]










  4%|▍         | 120545/2705037 [0

  6%|▌         | 166384/2705037 [00:26<06:12, 6806.51it/s]










  6%|▌         | 167069/2705037 [00:26<06:20, 6677.09it/s]










  6%|▌         | 167740/2705037 [00:26<06:21, 6655.93it/s]










  6%|▌         | 168408/2705037 [00:26<06:35, 6412.85it/s]










  6%|▋         | 169070/2705037 [00:26<06:31, 6473.38it/s]










  6%|▋         | 169776/2705037 [00:26<06:21, 6638.25it/s]










  6%|▋         | 170508/2705037 [00:26<06:11, 6828.96it/s]










  6%|▋         | 171195/2705037 [00:26<06:19, 6683.39it/s]










  6%|▋         | 171867/2705037 [00:26<06:25, 6572.69it/s]










  6%|▋         | 172527/2705037 [00:27<06:32, 6450.94it/s]










  6%|▋         | 173196/2705037 [00:27<06:28, 6519.30it/s]










  6%|▋         | 173860/2705037 [00:27<06:26, 6553.92it/s]










  6%|▋         | 174535/2705037 [00:27<06:22, 6610.64it/s]










  6%|▋         | 175258/2705037 [00:27<06:12, 6784.31it/s]










  7%|▋         | 176045/2705037 [0

  8%|▊         | 221615/2705037 [00:34<05:53, 7021.88it/s]










  8%|▊         | 222348/2705037 [00:34<05:49, 7109.23it/s]










  8%|▊         | 223061/2705037 [00:34<05:56, 6965.36it/s]










  8%|▊         | 223760/2705037 [00:34<06:09, 6707.53it/s]










  8%|▊         | 224435/2705037 [00:34<06:13, 6633.10it/s]










  8%|▊         | 225101/2705037 [00:34<06:17, 6569.07it/s]










  8%|▊         | 225772/2705037 [00:34<06:15, 6609.20it/s]










  8%|▊         | 226435/2705037 [00:35<06:17, 6569.58it/s]










  8%|▊         | 227093/2705037 [00:35<06:22, 6474.74it/s]










  8%|▊         | 227768/2705037 [00:35<06:18, 6553.50it/s]










  8%|▊         | 228484/2705037 [00:35<06:08, 6723.51it/s]










  8%|▊         | 229213/2705037 [00:35<05:59, 6883.46it/s]










  8%|▊         | 229926/2705037 [00:35<05:56, 6952.12it/s]










  9%|▊         | 230623/2705037 [00:35<06:10, 6680.17it/s]










  9%|▊         | 231347/2705037 [0

...

 93%|█████████▎| 2516140/2705037 [06:09<00:27, 6763.61it/s]










 93%|█████████▎| 2516818/2705037 [06:09<00:28, 6693.48it/s]










 93%|█████████▎| 2517489/2705037 [06:09<00:28, 6635.84it/s]










 93%|█████████▎| 2518176/2705037 [06:09<00:27, 6703.91it/s]










 93%|█████████▎| 2518885/2705037 [06:09<00:27, 6812.87it/s]










 93%|█████████▎| 2519568/2705037 [06:09<00:27, 6738.81it/s]










 93%|█████████▎| 2520297/2705037 [06:09<00:26, 6892.92it/s]










 93%|█████████▎| 2520988/2705037 [06:09<00:27, 6762.33it/s]










 93%|█████████▎| 2521666/2705037 [06:09<00:27, 6736.95it/s]










 93%|█████████▎| 2522360/2705037 [06:10<00:26, 6796.57it/s]










 93%|█████████▎| 2523041/2705037 [06:10<00:26, 6747.33it/s]










 93%|█████████▎| 2523723/2705037 [06:10<00:26, 6766.87it/s]










 93%|█████████▎| 2524401/2705037 [06:10<00:27, 6644.87it/s]










 93%|█████████▎| 2525082/2705037 [06:10<00:26, 6690.35it/s]










 93%|█████████▎| 252

 95%|█████████▌| 2572537/2705037 [06:17<00:18, 7103.45it/s]










 95%|█████████▌| 2573250/2705037 [06:17<00:18, 6959.75it/s]










 95%|█████████▌| 2573979/2705037 [06:17<00:18, 7052.96it/s]










 95%|█████████▌| 2574686/2705037 [06:17<00:18, 6900.24it/s]










 95%|█████████▌| 2575378/2705037 [06:17<00:18, 6864.05it/s]










 95%|█████████▌| 2576094/2705037 [06:17<00:18, 6948.11it/s]










 95%|█████████▌| 2576790/2705037 [06:17<00:18, 6852.54it/s]










 95%|█████████▌| 2577477/2705037 [06:17<00:18, 6778.68it/s]










 95%|█████████▌| 2578156/2705037 [06:18<00:18, 6714.83it/s]










 95%|█████████▌| 2578829/2705037 [06:18<00:18, 6691.29it/s]










 95%|█████████▌| 2579504/2705037 [06:18<00:18, 6708.72it/s]










 95%|█████████▌| 2580185/2705037 [06:18<00:18, 6737.05it/s]










 95%|█████████▌| 2580894/2705037 [06:18<00:18, 6838.75it/s]










 95%|█████████▌| 2581603/2705037 [06:18<00:17, 6910.76it/s]










 95%|█████████▌| 258

 97%|█████████▋| 2627991/2705037 [06:25<00:11, 6713.16it/s]










 97%|█████████▋| 2628681/2705037 [06:25<00:11, 6766.85it/s]










 97%|█████████▋| 2629359/2705037 [06:25<00:11, 6659.07it/s]










 97%|█████████▋| 2630027/2705037 [06:25<00:11, 6585.44it/s]










 97%|█████████▋| 2630721/2705037 [06:25<00:11, 6686.38it/s]










 97%|█████████▋| 2631391/2705037 [06:25<00:11, 6662.99it/s]










 97%|█████████▋| 2632059/2705037 [06:25<00:11, 6531.44it/s]










 97%|█████████▋| 2632740/2705037 [06:26<00:10, 6611.45it/s]










 97%|█████████▋| 2633482/2705037 [06:26<00:10, 6833.19it/s]










 97%|█████████▋| 2634168/2705037 [06:26<00:10, 6819.44it/s]










 97%|█████████▋| 2634852/2705037 [06:26<00:10, 6740.32it/s]










 97%|█████████▋| 2635559/2705037 [06:26<00:10, 6834.05it/s]










 97%|█████████▋| 2636255/2705037 [06:26<00:10, 6867.89it/s]










 97%|█████████▋| 2636943/2705037 [06:26<00:09, 6860.01it/s]










 98%|█████████▊| 263

 99%|█████████▉| 2683020/2705037 [06:33<00:03, 7019.62it/s]










 99%|█████████▉| 2683725/2705037 [06:33<00:03, 6985.85it/s]










 99%|█████████▉| 2684486/2705037 [06:33<00:02, 7159.43it/s]










 99%|█████████▉| 2685242/2705037 [06:33<00:02, 7274.86it/s]










 99%|█████████▉| 2685972/2705037 [06:33<00:02, 7066.05it/s]










 99%|█████████▉| 2686682/2705037 [06:33<00:02, 6942.25it/s]










 99%|█████████▉| 2687402/2705037 [06:34<00:02, 7015.06it/s]










 99%|█████████▉| 2688126/2705037 [06:34<00:02, 7080.25it/s]










 99%|█████████▉| 2688836/2705037 [06:34<00:02, 6953.30it/s]










 99%|█████████▉| 2689533/2705037 [06:34<00:02, 6789.28it/s]










 99%|█████████▉| 2690214/2705037 [06:34<00:02, 6720.23it/s]










 99%|█████████▉| 2690888/2705037 [06:34<00:02, 6633.38it/s]










100%|█████████▉| 2691584/2705037 [06:34<00:02, 6726.41it/s]










100%|█████████▉| 2692413/2705037 [06:34<00:01, 7128.02it/s]










100%|█████████▉| 269

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
2705007    None
2705008    None
2705009    None
2705010    None
2705011    None
2705012    None
2705013    None
2705014    None
2705015    None
2705016    None
2705017    None
2705018    None
2705019    None
2705020    None
2705021    None
2705022    None
2705023    None
2705024    None
2705025    None
2705026    None
2705027    None
2705028    None
2705029    None
2705030    None
2705031    None
2705032    None
2705033    None
2705034    None
2705035    None
2705036    None
Length: 2705037, dtype: 

In [124]:
lang_watch_times = {}
lang_watch_counts = {}

for user in user_lang_genre:

#     sum_val = sum(user_lang_genre[user].values())
    
    for lang in user_lang_genre[user]:    
        
        if lang not in lang_watch_times:
            lang_watch_times[lang] = user_lang_genre[user][lang]
            lang_watch_counts[lang] = 1
        else:
            lang_watch_times[lang] += user_lang_genre[user][lang]
            lang_watch_counts[lang] += 1        
        
print lang_watch_times

for lang in lang_watch_times:
    lang_watch_times[lang] /= lang_watch_counts[lang]

print lang_watch_times
    
for user in user_lang_genre:
    for lang in user_lang_genre[user]:    
        user_lang_genre[user][lang] /= lang_watch_times[lang]

{'oriya': 61381.0, 'sanskrit': 11790.0, 'telugu': 1900886449.0, 'turkish': 310560.0, 'manipuri': 4148.0, 'marathi': 139160855.0, 'punjabi': 90785.0, 'spanish': 2337361.0, 'romanian': 88688.0, 'kannada': 171764934.0, 'gujarati': 25547520.0, 'persian': 83568.0, 'hindi': 5688910539.0, 'konkani': 17258.0, 'english': 254105036.0, 'malayalam': 1534647134.0, 'rajasthani': 129152.0, 'tamil': 2140930629.0, 'bengali': 457994193.0}
{'oriya': 2273.3703703703704, 'sanskrit': 2947.5, 'telugu': 6407.604804810879, 'turkish': 2849.1743119266057, 'manipuri': 1037.0, 'marathi': 6076.893231441048, 'punjabi': 3631.4, 'spanish': 1936.5045567522784, 'romanian': 2771.5, 'kannada': 5490.328719833786, 'gujarati': 5253.448488587292, 'persian': 1671.36, 'hindi': 8523.644591843004, 'konkani': 4314.5, 'english': 5171.358365386573, 'malayalam': 9802.98266996276, 'rajasthani': 3075.0476190476193, 'tamil': 10948.923631212347, 'bengali': 7870.938904928851}


In [140]:
user_lang_genre

{'287144': {'action': 17706.0,
  'comedy': 827.0,
  'malayalam': 23510.0,
  'romance': 9121.0,
  'telugu': 4144.0},
 '287145': {'action': 14109.0,
  'hindi': 4703.0,
  'malayalam': 4703.0,
  'telugu': 4703.0},
 '287146': {'drama': 9547.0,
  'hindi': 3061.0,
  'tamil': 4324.0,
  'telugu': 2162.0},
 '378465': {'hindi': 6809.0, 'thriller': 6809.0},
 '378462': {'drama': 4989.0, 'hindi': 8022.0, 'thriller': 3033.0},
 '287141': {'comedy': 9600.0, 'drama': 1150.0, 'malayalam': 10750.0},
 '378460': {'action': 959.0,
  'drama': 3380.0,
  'hindi': 6254.0,
  'horror': 917.0,
  'malayalam': 2629.0,
  'thriller': 3627.0},
 '287143': {'action': 33621.0,
  'drama': 4844.0,
  'hindi': 7146.0,
  'malayalam': 21542.0,
  'romance': 2302.0,
  'telugu': 12079.0},
 '287148': {'action': 4595.0,
  'comedy': 982.0,
  'malayalam': 6832.0,
  'thriller': 1255.0},
 '378464': {'drama': 10158.0, 'malayalam': 10158.0},
 '378468': {'drama': 10820.0,
  'hindi': 2705.0,
  'tamil': 5410.0,
  'telugu': 2705.0},
 '378469':

In [116]:
user_lang_genre

{'287144': {'malayalam': 23510.0, 'telugu': 4144.0},
 '287145': {'hindi': 4703.0, 'malayalam': 4703.0, 'telugu': 4703.0},
 '287146': {'hindi': 3061.0, 'tamil': 4324.0, 'telugu': 2162.0},
 '378465': {'hindi': 6809.0},
 '378462': {'hindi': 8022.0},
 '287141': {'malayalam': 10750.0},
 '378460': {'hindi': 6254.0, 'malayalam': 2629.0},
 '287143': {'hindi': 7146.0, 'malayalam': 21542.0, 'telugu': 12079.0},
 '287148': {'malayalam': 6832.0},
 '378464': {'malayalam': 10158.0},
 '378468': {'hindi': 2705.0, 'tamil': 5410.0, 'telugu': 2705.0},
 '378469': {'hindi': 5194.0,
  'malayalam': 1046.0,
  'tamil': 8296.0,
  'telugu': 5194.0},
 '89370': {'bengali': 2931.0},
 '89371': {'gujarati': 9537.0},
 '89372': {'hindi': 15165.0},
 '89373': {'hindi': 35826.0},
 '89374': {'marathi': 750.0},
 '89375': {'telugu': 1084.0},
 '89376': {'hindi': 7891.0},
 '89377': {'telugu': 11333.0},
 '89378': {'hindi': 3998.0, 'tamil': 7996.0, 'telugu': 3998.0},
 '89379': {'hindi': 1420.0,
  'marathi': 819.0,
  'tamil': 2840

In [66]:
popular_lang_genre_movies

{'bengali_action': ['guru',
  'power',
  'josh',
  'boss',
  'yoddha: the warrior',
  'challenge 2',
  'champion',
  'pratibad',
  'tulkalam',
  'rocky',
  'surya',
  'macho mastana',
  'dada',
  'yuddha',
  'kistimaat',
  'gunda rajya',
  'mayer dibbi',
  'mujrim',
  'munna-ajob premer kahini'],
 'bengali_comedy': ['love express',
  'kelor kirti',
  'haripada bandwala',
  'jamai 420',
  'herogiri',
  'paglu 2',
  'paglu',
  'raja gaja no problem'],
 'bengali_crime': ['shatrudhwanso'],
 'bengali_drama': ['zulfiqar',
  'rajkahini',
  'deewana',
  'sathi',
  'krishna',
  'bela seshe',
  'sedhin dekha hoyechilo',
  'chokher bali',
  'amanush 2',
  'shankhachil',
  'chander pahar',
  'rangbaaz',
  'jekhane bhooter bhoy',
  'loveria',
  'challenge',
  'sasurbari zindabad',
  'dui prithibi',
  'mishawr rawhoshyo',
  'amanush',
  'satyanweshi'],
 'bengali_family': ['bal ganesh',
  'ghatothkach (bengali)',
  'return of hanuman (bengali)',
  'lal kalo (cheenti cheenti bang bang)',
  'badhu',
  

In [141]:
res = {}

def get_similar_items(row):
        
#     rows_lang_genre = {}
    res_list = []
    
#     counter = 0
    
#     for lang_genre in [i[0] for i in sorted(user_lang_genre[str(row.user_id)].items(), key=operator.itemgetter(1))]:
        
#         counter += 1
        
#         if counter == len(user_lang_genre[str(row.user_id)]):
#             num_rows = 20 - len(res_list)
#         else:
#             num_rows = int(user_lang_genre[str(row.user_id)][lang_genre]*20)

#         res_list = res_list + popular_lang_genre_movies[lang_genre][:num_rows]
        
    sorted_langs = [i[0] for i in sorted(user_lang_genre[str(row.user_id)].items(), key=operator.itemgetter(1), reverse = True)]
    
#     most_watched_lang_genre = max(user_lang_genre[str(row.user_id)].iteritems(), key=operator.itemgetter(1))[0]

    for lang in sorted_langs:
        
        if len(res_list) < 20:
            res_list += popular_lang_genre_movies[lang][:20 - len(res_list)]
    
    if len(res_list) < 20:
        res_list += popular_lang_genre_movies["hindi"][:20]
        
    res[str(row.user_id)] = list(set(res_list))[:20]
        
#     res[str(row.user_id)] = list(set(res_list))
    
test.progress_apply(get_similar_items, axis = 1)












  0%|          | 0/6285 [00:00<?, ?it/s]










 13%|█▎        | 841/6285 [00:00<00:00, 8408.06it/s]










 27%|██▋       | 1701/6285 [00:00<00:00, 8461.89it/s]










 39%|███▉      | 2470/6285 [00:00<00:00, 8201.02it/s]










 48%|████▊     | 3004/6285 [00:00<00:00, 6944.38it/s]










 72%|███████▏  | 4510/6285 [00:00<00:00, 8283.44it/s]










 93%|█████████▎| 5829/6285 [00:00<00:00, 9323.10it/s]










100%|██████████| 6285/6285 [00:00<00:00, 9772.58it/s]

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
6255    None
6256    None
6257    None
6258    None
6259    None
6260    None
6261    None
6262    None
6263    None
6264    None
6265    None
6266    None
6267    None
6268    None
6269    None
6270    None
6271    None
6272    None
6273    None
6274    None
6275    None
6276    None
6277    None
6278    None
6279    None
6280    None
6281    None
6282    None
6283    None
6284    None
Length: 6285, dtype: object

In [143]:
import json
with open('res_watch_time_sum_lang_genre.json', 'w') as fp:
    json.dump(res, fp)

In [145]:
np.mean([len(res[i]) for i in res])

20.0

In [144]:
res

{'172799': ['pink',
  'bedroom partner',
  'sholay',
  'bajrangi bhaijaan',
  'the dirty picture',
  'akira',
  'talvar',
  'julie',
  'ms dhoni: the untold story',
  'rathinirvedam',
  'jaggu dada',
  'jolly llb 2',
  'nishabd',
  'oozham',
  's/o satyamurthy',
  'bhaag milkha bhaag',
  'ghayal',
  'neerja',
  'sarkar raj',
  'highway'],
 '431071': ['love express',
  'jolly llb',
  'marubhoomiyile aana',
  'andaz apna apna',
  'kelor kirti',
  'tere bin laden 2',
  'khatta meetha',
  'aadu',
  'chhello divas',
  'haripada bandwala',
  'golmaal 3',
  'bhagam bhag',
  'housefull 2',
  'housefull 3',
  'dolly ki doli',
  'bade miyan chote miyan',
  'yaariyan',
  'adi kapyare kootamani',
  'besharam',
  '2 countries'],
 '338331': ['daagdi chawl',
  'holiday',
  'khaidi no. 150',
  'kabali',
  'kammattipadam',
  'baahubali - the beginning',
  'pulimurugan',
  'guru',
  'sardar gabbar singh',
  'maari',
  'dishoom',
  'singham returns',
  'janatha garage',
  'bang bang',
  'action jackson',